In [2]:
import http.client
import pandas as pd
import json
import time
from pandas.io.json import json_normalize


# 1º. Looking for the seasons ID with the seasons of Six Nations

In [19]:
conn_season = http.client.HTTPSConnection("api.sportradar.us")

conn_season.request("GET", "/rugby/trial/v2/union/en/seasons.json?api_key=az4p5r34nkkuf85ax9jefx92")

res_season = conn_season.getresponse()
raw_season = res_season.read()



In [101]:
raw_season

season = raw_season.decode("utf-8")
season_json = json.loads(season)

data_seasons = json_normalize(season_json)
data_seasons['seasons'][0]
# take in hand the id

[{'id': 'sr:season:9032',
  'name': 'British and Irish Cup, Q 2014',
  'start_date': '2014-08-30',
  'end_date': '2014-09-07',
  'year': '2014',
  'tournament_id': 'sr:tournament:1426',
  'sport': {'id': 'sr:sport:12', 'name': 'Rugby'},
  'category': {'id': 'sr:category:82', 'name': 'Rugby Union'},
  'competition': {'id': 'sr:tournament:1426',
   'name': 'British and Irish Cup, Q'}},
 {'id': 'sr:season:10018',
  'name': 'Vodacom Cup 2015',
  'start_date': '2015-03-06',
  'end_date': '2015-05-31',
  'year': '2015',
  'tournament_id': 'sr:tournament:1272',
  'sport': {'id': 'sr:sport:12', 'name': 'Rugby'},
  'category': {'id': 'sr:category:82', 'name': 'Rugby Union'},
  'competition': {'id': 'sr:tournament:1272', 'name': 'Vodacom Cup'}},
 {'id': 'sr:season:10249',
  'name': 'World Rugby Tbilisi Cup 2015',
  'start_date': '2015-06-13',
  'end_date': '2015-06-22',
  'year': '2015',
  'tournament_id': 'sr:tournament:1740',
  'sport': {'id': 'sr:sport:12', 'name': 'Rugby'},
  'category': {'i

# 2º Making the call for the season to find the teams

#### Doing the call for each season

In [27]:
x = 'sr:season:69388'

# id of Six Nations por ahora

conn_team = http.client.HTTPSConnection("api.sportradar.us")
conn_team.request("GET", f"/rugby/trial/v2/union/en/seasons/{x}/info.json?api_key=az4p5r34nkkuf85ax9jefx92")

res_team = conn_team.getresponse()
raw_team = res_team.read()
raw_team

team = raw_team.decode("utf-8")
team_json = json.loads(team)

data_team = json_normalize(team_json)
data_team


,generated_at,schema,stages,season.id,season.name,season.start_date,season.end_date,season.year,season.tournament_id,season.sport.id,season.sport.name,season.category.id,season.category.name,season.competition.id,season.competition.name,coverage_info.live_coverage
0,2019-11-26T17:26:00+00:00,http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'number': 1, 'type': 'league', 'start_date':...",sr:season:69388,Six Nations 2020,2020-02-01,2020-03-15,2020,sr:tournament:423,sr:sport:12,Rugby,sr:category:82,Rugby Union,sr:tournament:423,Six Nations,True


In [105]:
data_group

,id,name,competitors,max_rounds
0,sr:group:47839,Six Nations 2020,"[{'id': 'sr:competitor:4224', 'name': 'France'...",5


In [28]:
data_team2 = json_normalize(data_team['stages'][0])
data_group = json_normalize(data_team2['groups'][0])
competitors = data_group['competitors'][0]
competitors

[{'id': 'sr:competitor:4224',
  'name': 'France',
  'country': 'France',
  'country_code': 'FRA',
  'abbreviation': 'FRA'},
 {'id': 'sr:competitor:7058',
  'name': 'Wales',
  'country': 'Wales',
  'country_code': 'WAL',
  'abbreviation': 'WAL'},
 {'id': 'sr:competitor:4226',
  'name': 'England',
  'country': 'England',
  'country_code': 'ENG',
  'abbreviation': 'ENG'},
 {'id': 'sr:competitor:4228',
  'name': 'Ireland',
  'country': 'Ireland',
  'country_code': 'IRL',
  'abbreviation': 'IRL'},
 {'id': 'sr:competitor:4230',
  'name': 'Scotland',
  'country': 'Scotland',
  'country_code': 'SCO',
  'abbreviation': 'SCO'},
 {'id': 'sr:competitor:7057',
  'name': 'Italy',
  'country': 'Italy',
  'country_code': 'ITA',
  'abbreviation': 'ITA'}]

In [29]:
competitors_id = []
for team in competitors:
    competitors_id.append(team['id'])
print(competitors_id)

['sr:competitor:4224', 'sr:competitor:7058', 'sr:competitor:4226', 'sr:competitor:4228', 'sr:competitor:4230', 'sr:competitor:7057']


In [31]:
df_competitors = pd.DataFrame()
for x in competitors_id:
    
    time.sleep(1.2)
    conn = http.client.HTTPSConnection("api.sportradar.us")

    conn.request("GET", f"/rugby/trial/v2/union/en/teams/{x}/profile.json?api_key=az4p5r34nkkuf85ax9jefx92")

    res = conn.getresponse()
    data = res.read()
    
    a = data.decode("utf-8")
    data_json = json.loads(a)
    
    data1 = json_normalize(data_json)
    df_competitors = df_competitors.append(data1)
    
df_competitors

,competitor.abbreviation,competitor.category.id,competitor.category.name,competitor.country,competitor.country_code,competitor.id,competitor.name,competitor.sport.id,competitor.sport.name,generated_at,...,players,schema,statistics.seasons,venue,venue.capacity,venue.city_name,venue.country_code,venue.country_name,venue.id,venue.name
0,FRA,sr:category:82,Rugby Union,France,FRA,sr:competitor:4224,France,sr:sport:12,Rugby,2019-11-26T17:27:12+00:00,...,"[{'id': 'sr:player:428238', 'name': 'Medard, M...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:1303', 'name': 'Six Nations...",[],NaN,NaN,NaN,NaN,NaN,NaN
0,WAL,sr:category:82,Rugby Union,Wales,WAL,sr:competitor:7058,Wales,sr:sport:12,Rugby,2019-11-26T17:27:14+00:00,...,"[{'id': 'sr:player:447024', 'name': 'Jones, Al...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:1303', 'name': 'Six Nations...",[],NaN,NaN,NaN,NaN,NaN,NaN
0,ENG,sr:category:82,Rugby Union,England,ENG,sr:competitor:4226,England,sr:sport:12,Rugby,2019-11-26T17:27:16+00:00,...,"[{'id': 'sr:player:374332', 'name': 'Daly, Ell...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:1303', 'name': 'Six Nations...",NaN,82000.0,London,ENG,England,sr:venue:16049,Twickenham Stadium
0,IRL,sr:category:82,Rugby Union,Ireland,IRL,sr:competitor:4228,Ireland,sr:sport:12,Rugby,2019-11-26T17:27:18+00:00,...,"[{'id': 'sr:player:440570', 'name': 'Best, Ror...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:1303', 'name': 'Six Nations...",[],NaN,NaN,NaN,NaN,NaN,NaN
0,SCO,sr:category:82,Rugby Union,Scotland,SCO,sr:competitor:4230,Scotland,sr:sport:12,Rugby,2019-11-26T17:27:19+00:00,...,"[{'id': 'sr:player:398650', 'name': 'Barclay, ...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:1303', 'name': 'Six Nations...",[],NaN,NaN,NaN,NaN,NaN,NaN
0,ITA,sr:category:82,Rugby Union,Italy,ITA,sr:competitor:7057,Italy,sr:sport:12,Rugby,2019-11-26T17:27:21+00:00,...,"[{'id': 'sr:player:410882', 'name': 'Parisse, ...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:1303', 'name': 'Six Nations...",[],NaN,NaN,NaN,NaN,NaN,NaN


# Making the list for the players

In [56]:
players = pd.DataFrame(df_competitors['players'])

#Reset index
players = players.reset_index(drop=True)

#get the list of players id:
players_id=[]

for x in range(len(players)):
    
    for y in players['players'][x]:
        
         players_id.append(y['id'])
            
print(players_id)

['sr:player:428238', 'sr:player:511834', 'sr:player:1325424', 'sr:player:1146768', 'sr:player:1068550', 'sr:player:955152', 'sr:player:952348', 'sr:player:939646', 'sr:player:801908', 'sr:player:768562', 'sr:player:765318', 'sr:player:548996', 'sr:player:535082', 'sr:player:512794', 'sr:player:512726', 'sr:player:512038', 'sr:player:511818', 'sr:player:450872', 'sr:player:511702', 'sr:player:511646', 'sr:player:504958', 'sr:player:504180', 'sr:player:502228', 'sr:player:498388', 'sr:player:487822', 'sr:player:470570', 'sr:player:470568', 'sr:player:470564', 'sr:player:464304', 'sr:player:464296', 'sr:player:459964', 'sr:player:454684', 'sr:player:1645924', 'sr:player:447024', 'sr:player:491382', 'sr:player:534552', 'sr:player:534530', 'sr:player:512914', 'sr:player:512834', 'sr:player:512122', 'sr:player:507010', 'sr:player:500802', 'sr:player:497708', 'sr:player:497652', 'sr:player:497296', 'sr:player:497082', 'sr:player:496506', 'sr:player:495358', 'sr:player:488394', 'sr:player:4513

# Making the DataFrame for the players

In [59]:
players_df = pd.DataFrame()
for x in players_id:
    
    time.sleep(1.2)
    conn = http.client.HTTPSConnection("api.sportradar.us")

    conn.request("GET", f"/rugby/trial/v2/union/en/players/{x}/profile.json?api_key=az4p5r34nkkuf85ax9jefx92")

    res = conn.getresponse()
    data = res.read()
    time.sleep(1)
    a = data.decode("utf-8")
    data_json = json.loads(a)
    
    data1 = json_normalize(data_json)
    players_df = players_df.append(data1)
    
players_df

,competitors,generated_at,player.country_code,player.date_of_birth,player.first_name,player.gender,player.height,player.id,player.last_name,player.name,player.nationality,player.nickname,player.type,player.weight,roles,schema,statistics.seasons
0,"[{'id': 'sr:competitor:4217', 'name': 'Stade T...",2019-11-27T08:00:29+00:00,FRA,1986-11-16,Maxime,male,182,sr:player:428238,Medard,"Medard, Maxime",France,NaN,W,87,"[{'type': 'player', 'active': 'true', 'competi...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:56025', 'name': 'European C..."
0,"[{'id': 'sr:competitor:4224', 'name': 'France'...",2019-11-27T08:00:33+00:00,FRA,1994-03-26,Gael,male,190,sr:player:511834,Fickou,"Fickou, Gael",France,NaN,C,100,"[{'type': 'player', 'active': 'false', 'compet...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:10277', 'name': 'World Cup ..."
0,"[{'id': 'sr:competitor:4224', 'name': 'France'...",2019-11-27T08:00:35+00:00,FRA,1997-03-23,Gregory,male,191,sr:player:1325424,Alldritt,"Alldritt, Gregory",France,NaN,BR,111,"[{'type': 'player', 'active': 'true', 'competi...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:59616', 'name': 'World Cup ..."
0,"[{'id': 'sr:competitor:4217', 'name': 'Stade T...",2019-11-27T08:00:40+00:00,FRA,1999-05-01,Romain,male,186,sr:player:1146768,N'tamack,"N'tamack, Romain",France,NaN,FH,81,"[{'type': 'player', 'active': 'true', 'competi...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:56025', 'name': 'European C..."
0,"[{'id': 'sr:competitor:4224', 'name': 'France'...",2019-11-27T08:00:43+00:00,FRA,1998-04-22,Pierre-Louis,male,188,sr:player:1068550,Barassi,"Barassi, Pierre-Louis",France,NaN,C,92,"[{'type': 'player', 'active': 'true', 'competi...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:56025', 'name': 'European C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"[{'id': 'sr:competitor:7057', 'name': 'Italy',...",2019-11-27T08:09:36+00:00,ITA,1987-12-11,Jayden,male,184,sr:player:474312,Hayward,"Hayward, Jayden",Italy,Jayden,FB,90,"[{'type': 'player', 'active': 'false', 'compet...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:55971', 'name': 'Pro 14 18/..."
0,"[{'id': 'sr:competitor:7057', 'name': 'Italy',...",2019-11-27T08:09:39+00:00,ITA,1986-07-31,Dean,male,196,sr:player:463080,Budd,"Budd, Dean",Italy,NaN,L,100,"[{'type': 'player', 'active': 'false', 'compet...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:10844', 'name': 'Pro 12 15/..."
0,"[{'id': 'sr:competitor:7057', 'name': 'Italy',...",2019-11-27T08:09:42+00:00,ITA,1987-09-23,Tito,male,180,sr:player:460468,Tebaldi,"Tebaldi, Tito",Italy,NaN,SH,88,"[{'type': 'player', 'active': 'false', 'compet...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:10620', 'name': 'English Pr..."
0,"[{'id': 'sr:competitor:7057', 'name': 'Italy',...",2019-11-27T08:09:45+00:00,ITA,1984-01-31,Alessandro,male,193,sr:player:450864,Zanni,"Zanni, Alessandro",Italy,NaN,L,111,"[{'type': 'player', 'active': 'false', 'compet...",http://schemas.sportradar.com/bsa/rugby/v1/jso...,"[{'id': 'sr:season:10277', 'name': 'World Cup ..."


# Making the table for red and yellow cards

In [88]:
df_RY = pd.DataFrame()

def role(team_df):
    for active in range(len(team_df)):
        if team_df.loc[active,'competitor.name'] in  ['France','Scotland','Italy','Ireland','England','Wales']:
            return team_df.loc[active,'competitor.name']



#
for idx in range(len(players_df)):
    statistics = players_df.loc[idx, 'statistics.seasons']
    team = players_df.loc[idx, 'roles']
    
    #Making the 
    columns_to_keep = ['id','name','statistics.yellow_cards','statistics.red_cards']
    statistics_df = json_normalize(statistics)[columns_to_keep]
    statistics_df['Player_Name'] = players_df.loc[idx, 'player.name']
    statistics_df['Player_id'] = players_df.loc[idx, 'player.id']
    statistics_df['Position'] = players_df.loc[idx, 'player.type']
    statistics
    #Input the team to the DF
    team_df = json_normalize(team)
    statistics_df['Team'] = role(team_df)
    
    #Append the DF to another DF with nothing
    df_RY = df_RY.append(statistics_df)

    
#Rename the columns with a more sense name.    
df_RY = df_RY.rename(columns={ 'id':'Season_id','name': 'Season_name', 'statistics.yellow_cards': 'Yellow_Cards',\
                             'statistics.red_cards': 'Red_Cards'})    

#Relocated the columns in the order I want
df_RY = df_RY[['Player_Name','Player_id','Position','Team','Season_id','Season_name','Yellow_Cards','Red_Cards']]

df_RY = df_RY.reset_index(drop=True)
df_RY


,Player_Name,Player_id,Position,Team,Season_id,Season_name,Yellow_Cards,Red_Cards
0,"Medard, Maxime",sr:player:428238,W,France,sr:season:56025,European Cup 18/19,0,0
1,"Medard, Maxime",sr:player:428238,W,France,sr:season:35125,European Cup 16/17,0,0
2,"Medard, Maxime",sr:player:428238,W,France,sr:season:59616,World Cup 2019,0,0
3,"Medard, Maxime",sr:player:428238,W,France,sr:season:10718,Top 14 15/16,0,0
4,"Medard, Maxime",sr:player:428238,W,France,sr:season:11032,Six Nations 2016,0,0
...,...,...,...,...,...,...,...,...
1790,"Zanni, Alessandro",sr:player:450864,L,Italy,sr:season:59616,World Cup 2019,0,0
1791,"Riccioni, Marco",sr:player:942226,PR,Italy,sr:season:45148,European Cup 17/18,0,0
1792,"Riccioni, Marco",sr:player:942226,PR,Italy,sr:season:55971,Pro 14 18/19,0,0
1793,"Riccioni, Marco",sr:player:942226,PR,Italy,sr:season:67705,Pro 14 19/20,0,0


In [98]:
df_RY.Season_name.unique()

array(['European Cup 18/19', 'European Cup 16/17', 'World Cup 2019',
       'Top 14 15/16', 'Six Nations 2016', 'European Cup 15/16',
       'Six Nations 2019', 'European Cup 19/20', 'World Cup 2015',
       'Six Nations 2017', 'Six Nations 2018', 'European Cup 17/18',
       'English Premiership 16/17', 'Pro 14 17/18', 'Pro 12 16/17',
       'Pro 14 18/19', 'Pro 12 15/16', 'Pro 14 19/20',
       'English Premiership 15/16', 'English Premiership 17/18',
       'English Premiership 18/19', 'Premiership Rugby Cup 18/19',
       'English Premiership 19/20', 'Premiership Rugby Cup 19/20',
       'Super Rugby 2016', 'Super Rugby 2017', 'Super Rugby 2018',
       'English A League 2015'], dtype=object)

#### Create the columns for Back and Forwards to been able to categorize the players position by adding a new column in the DataFrame

In [107]:
def field_position(c):
    if c['Position'] in ['L', 'HO', 'PR', 'BR']:
        return 'Forward'
    elif c['Position'] in ['FH', 'SH', 'C', 'W', 'FB']:
        return 'Back'

df_RY['Field Position'] = df_RY.apply(field_position, axis=1)    

df_RY

,Player_Name,Player_id,Position,Team,Season_id,Season_name,Yellow_Cards,Red_Cards,Field Position
0,"Medard, Maxime",sr:player:428238,W,France,sr:season:56025,European Cup 18/19,0,0,Back
1,"Medard, Maxime",sr:player:428238,W,France,sr:season:35125,European Cup 16/17,0,0,Back
2,"Medard, Maxime",sr:player:428238,W,France,sr:season:59616,World Cup 2019,0,0,Back
3,"Medard, Maxime",sr:player:428238,W,France,sr:season:10718,Top 14 15/16,0,0,Back
4,"Medard, Maxime",sr:player:428238,W,France,sr:season:11032,Six Nations 2016,0,0,Back
...,...,...,...,...,...,...,...,...,...
1790,"Zanni, Alessandro",sr:player:450864,L,Italy,sr:season:59616,World Cup 2019,0,0,Forward
1791,"Riccioni, Marco",sr:player:942226,PR,Italy,sr:season:45148,European Cup 17/18,0,0,Forward
1792,"Riccioni, Marco",sr:player:942226,PR,Italy,sr:season:55971,Pro 14 18/19,0,0,Forward
1793,"Riccioni, Marco",sr:player:942226,PR,Italy,sr:season:67705,Pro 14 19/20,0,0,Forward


# Creating the JSON File for analysis

In [109]:
df_RY.to_csv('Data/DATA_RY_CARDS.csv')